In [1]:
import os
import time
import numpy as np
import faiss
import openai

# OpenAI API 키 설정
openai.api_key = 'sk-proj-kWMWHOar0E4DdGmIHd9VT3BlbkFJ1sODyQZo64LaHKl2HYrL'

# 현재 디렉토리 설정
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, 'Data')
vector_db_dir = os.path.join(current_dir, 'VectorDB')

# VectorDB 디렉토리가 없으면 생성
if not os.path.exists(vector_db_dir):
    os.makedirs(vector_db_dir)

# 모든 txt 파일 읽어오기
txt_files = [f for f in os.listdir(data_dir) if f.endswith('.txt')]

# 모든 텍스트 파일의 내용을 리스트로 저장
documents = []
file_paths = []

for txt_file in txt_files:
    txt_file_path = os.path.join(data_dir, txt_file)
    
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        documents.append(text)
        file_paths.append(txt_file_path)

# 임베딩 생성 함수
def get_embedding(text):
    response = openai.Embedding.create(
        input=text,
        engine="text-embedding-ada-002"
    )
    return np.array(response['data'][0]['embedding'], dtype=np.float32)

# 모든 문서에 대해 임베딩 생성
embeddings = []
for i, doc in enumerate(documents):
    embeddings.append(get_embedding(doc))
    if (i + 1) % 3 == 0:
        time.sleep(60)  # 3회 요청 후 1분 대기

embeddings = np.array(embeddings)

# FAISS 인덱스 생성
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# 인덱스를 저장
faiss.write_index(index, os.path.join(vector_db_dir, 'vector_db.index'))

# 파일 경로 저장
with open(os.path.join(vector_db_dir, 'file_paths.txt'), 'w', encoding='utf-8') as f:
    for path in file_paths:
        f.write(path + '\n')

print("모든 txt 파일을 벡터화하여 VectorDB 폴더에 저장했습니다.")

# RAG 검색 함수
def search(query, top_k=5):
    # 쿼리 벡터화
    query_embedding = get_embedding(query).reshape(1, -1)
    
    # FAISS 인덱스에서 유사한 문서 검색
    D, I = index.search(query_embedding, top_k)
    
    # 검색된 문서 경로 가져오기
    with open(os.path.join(vector_db_dir, 'file_paths.txt'), 'r', encoding='utf-8') as f:
        file_paths = f.readlines()
    
    results = [file_paths[i].strip() for i in I[0]]
    return results

# 예시 질문
query = "예시 질문을 입력하세요"
result_files = search(query)

print("검색 결과 파일 경로:", result_files)


모든 txt 파일을 벡터화하여 VectorDB 폴더에 저장했습니다.
검색 결과 파일 경로: ['c:\\Users\\wonhy\\OneDrive\\Desktop\\gist-policy-bot\\Data\\FR00602 여비규칙_국내출장비.txt', 'c:\\Users\\wonhy\\OneDrive\\Desktop\\gist-policy-bot\\Data\\FR00602 여비규칙_1.txt', 'c:\\Users\\wonhy\\OneDrive\\Desktop\\gist-policy-bot\\Data\\FR00602 여비규칙_국외출장비.txt', 'c:\\Users\\wonhy\\OneDrive\\Desktop\\gist-policy-bot\\Data\\FR00602 여비규칙_보칙.txt', 'c:\\Users\\wonhy\\OneDrive\\Desktop\\gist-policy-bot\\Data\\FR00602 여비규칙_보칙.txt']


In [3]:
# 예시 검색
query = "예시 질문"
results = search(query)
print("검색 결과:", results)

검색 결과: ['c:\\Users\\wonhy\\OneDrive\\Desktop\\gist-policy-bot\\Data\\FR00602 여비규칙_국내출장비.txt', 'c:\\Users\\wonhy\\OneDrive\\Desktop\\gist-policy-bot\\Data\\FR00602 여비규칙_보칙.txt', 'c:\\Users\\wonhy\\OneDrive\\Desktop\\gist-policy-bot\\Data\\FR00602 여비규칙_국외출장비.txt', 'c:\\Users\\wonhy\\OneDrive\\Desktop\\gist-policy-bot\\Data\\FR00602 여비규칙_1.txt', 'c:\\Users\\wonhy\\OneDrive\\Desktop\\gist-policy-bot\\Data\\FR00602 여비규칙_보칙.txt']
